In [24]:
# Navier–Stokes 2D (FEM, NGSolve) – Kanalströmung um Zylinder mit Wirbelstraße
# Robust gegen "alles = 0" durch:
#   - Druck-Fixierung: p=0 am Outlet (Q = H1 mit dirichlet="outlet")
#   - korrekte Konvektion: (u_old·∇)u  = grad(u)*u_old
#   - Dirichlet nur über FESpace (inlet|walls|cyl), Inflow einmalig gesetzt
#   - kleiner Symmetriebruch am Anfang

from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.webgui import Draw
import math


In [25]:
# -----------------------------
# 1) Geometrie + Mesh (Kanal minus Zylinder)
# -----------------------------
L, H = 2.2, 0.41
cx, cy, r = 0.2, 0.2, 0.05

geo = SplineGeometry()

p1 = geo.AddPoint(0, 0)
p2 = geo.AddPoint(L, 0)
p3 = geo.AddPoint(L, H)
p4 = geo.AddPoint(0, H)

geo.AddSegment([p1, p2], bc="walls")     # unten
geo.AddSegment([p2, p3], bc="outlet")    # rechts
geo.AddSegment([p3, p4], bc="walls")     # oben
geo.AddSegment([p4, p1], bc="inlet")     # links

# Zylinderloch
geo.AddCircle((cx, cy), r=r, bc="cyl", leftdomain=0, rightdomain=1)

mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw(mesh)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [26]:
# -----------------------------
# 2) FEM-Räume
#   u in VectorH1 (Taylor–Hood), Dirichlet: inlet|walls|cyl
#   p in H1 und Dirichlet: outlet  -> fixiert Druckniveau (sonst singulär)
# -----------------------------
order_u = 2
order_p = 1

V = VectorH1(mesh, order=order_u, dirichlet="inlet|walls|cyl")
Q = H1(mesh, order=order_p, dirichlet="outlet")   # p=0 am outlet
X = V * Q

(u, p) = X.TrialFunction()
(v, q) = X.TestFunction()

gfu = GridFunction(X)
un, pn = gfu.components


In [27]:
# -----------------------------
# 3) Parameter + Inflow
# -----------------------------
from ngsolve import x, y
uold = GridFunction(V)


Re = 100.0
D  = 2*r
Umax = 1.0
nu = Umax*D/Re

dt = 0.002
t_end = 2.0

# Parabolisches Inflow-Profil + kleine Störung (Symmetriebruch!)
Umean = Umax / 1.5
uin = CoefficientFunction((
    4*Umean*y*(H-y)/(H*H) * (1 + 0.01*sin(20*y)),
    0
))

# Randbedingungen
un.Set(uin, definedon=mesh.Boundaries("inlet"))
un.Set((0,0), definedon=mesh.Boundaries("walls|cyl"))
pn.Set(0, definedon=mesh.Boundaries("outlet"))

# Initialwert
uold.vec.data = un.vec


In [28]:
# -----------------------------
# 4) Zeitdiskretisierung (semi-implicit Oseen):
#    (u^{n+1}-u^n)/dt + (u^n·∇)u^{n+1} - nu Δu^{n+1} + ∇p^{n+1} = 0
#    div u^{n+1} = 0
# -----------------------------
# ACHTUNG: uold NICHT neu anlegen, sondern aus vorheriger Zelle übernehmen!
# KEIN: uold = GridFunction(V)
#
scene_u = Draw(un, mesh, "velocity")
scene_p = Draw(pn, mesh, "pressure")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [29]:
# -----------------------------
# 5) Zeitschleife
# -----------------------------
t = 0.0
step = 0

while t < t_end - 1e-12:

    # Bilinearform (abhängig von uold -> neu assemblieren)
    a = BilinearForm(X, symmetric=False)
    a += (1/dt) * InnerProduct(u, v) * dx
    a += nu * InnerProduct(grad(u), grad(v)) * dx
    a += InnerProduct(grad(u) * uold, v) * dx          # (uold·∇)u
    a += (-div(v)*p - div(u)*q) * dx                    # Inkompressibilität
    a.Assemble()

    # RHS
    f = LinearForm(X)
    f += (1/dt) * InnerProduct(uold, v) * dx
    f.Assemble()

    # Solve (direkt, robust)
    inv = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")
    gfu.vec.data = inv * f.vec

    # Update
    uold.vec.data = un.vec
    t += dt
    step += 1

    if step % 20 == 0:
        scene_u.Redraw()
        scene_p.Redraw()

t, step


(2.0000000000000013, 1000)

In [30]:
# -----------------------------
# 6) Wirbelstärke (Vorticity) für VectorH1:
#    omega = d(uy)/dx - d(ux)/dy = grad(u)[1,0] - grad(u)[0,1]
# -----------------------------
G = grad(un)
omega = G[1,0] - G[0,1]
Draw(omega, mesh, "vorticity")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene